## Topic Modelling -  Coursera

In [1]:
import sys

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from pprint import pprint

# Plotting tools
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
path = '--'
%run ./Text_Normalization_Function.ipynb

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [3]:
df = pd.read_csv(path+"Coursera_txt.txt", delimiter = "\t", header = None)

In [4]:
# remove http, and @ and username
for index, row in df.iterrows():
    row[0] = str(row[0]).split(" ")
    for i in range(len(row[0])):
        row[0][i] = "".join([word for word in row[0][i].split()
                            if "http" not in word and "@" not in word and "<" not in word])
    row[0] = ",".join(row[0])
    row[0] = str(row[0]).replace(",", " ")
    

**** Set Up the Topic Model (LDA) ****

In [5]:
normalized_coursera = normalize_corpus(df[0])
bow_vectorizer_coursera = CountVectorizer()
bow_coursera_corpus = bow_vectorizer_coursera.fit_transform(normalized_coursera)
bow_feature_names_coursera = bow_vectorizer_coursera.get_feature_names()
bow_table = pd.DataFrame(data = bow_coursera_corpus.todense(), columns = bow_feature_names_coursera)
bow_table.head(5)

,0m,100daysofcode,100daysofmlcode,100daysofmlcodestudying,100finished,103mfor,10kwomen,1m,4th,5m,...,yeah,year,yearup,yesterday,york,youll,youre,yourselfshun,yourstoryco,youtube
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# set up the model 
no_topics = 5
lda_coursera_corpus = LatentDirichletAllocation(n_components=no_topics, 
                                           doc_topic_prior =0.3,
                                           topic_word_prior = 0.2).fit(bow_coursera_corpus)


C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [7]:
#Results
no_top_words = 10
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words
topic_words = get_topic_words(vectorizer = bow_vectorizer_coursera, 
                              lda_model = lda_coursera_corpus, 
                              n_words = no_top_words)
pd.DataFrame(topic_words, 
             columns = ["word_" + str(i) for i in range(no_top_words)],
             index = ["Topic_" + str(i) for i in range(len(topic_words))]) 

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
Topic_0,learn,online,startup,value,pick,series,round,fund,valuation,today
Topic_1,course,free,astrobiology,cool,colleague,arizona,skill,take,data,mooc
Topic_2,business,like,datascience,education,10kwomen,create,right,learning,entrepreneur,lead
Topic_3,first,time,campus,long,revision,overdue,wit,remember,maths,smith
Topic_4,online,raise,million,learner,fourth,prepare,revolution,industrial,billion,education


In [8]:
#Display a word vectors (words are in alphabetical order) for each topic
word_weights = lda_coursera_corpus.components_ / lda_coursera_corpus.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T, 
                               index = bow_feature_names_coursera, 
                               columns = ["Topic_" + str(i) for i in range(no_topics)])
word_weights_df.head(10)

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
0m,0.000470,0.000084,0.000138,0.000164,0.000154
100daysofcode,0.000090,0.000084,0.000140,0.000164,0.008559
100daysofmlcode,0.000089,0.000125,0.003445,0.000164,0.000167
100daysofmlcodestudying,0.000088,0.000084,0.000138,0.000164,0.001427
100finished,0.000089,0.000084,0.001990,0.000164,0.000212
103mfor,0.000088,0.000084,0.000735,0.000163,0.000154
10kwomen,0.000089,0.000084,0.021138,0.000163,0.000175
1m,0.000089,0.000446,0.000138,0.000164,0.000155
4th,0.000088,0.000084,0.000138,0.000164,0.001427
5m,0.000088,0.000446,0.000138,0.000164,0.000155


In [9]:
#sort by word weights in Topic 0 (descending order) and see the weights by 10 most frequent words in Topic 0:
word_weights_df.sort_values(by='Topic_0',ascending=False).head(10)

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
learn,0.116600,0.000087,0.000151,0.000164,0.000200
online,0.110010,0.000088,0.000174,0.000178,0.068615
startup,0.107584,0.000084,0.000139,0.000164,0.000753
value,0.086035,0.000084,0.000138,0.000164,0.000154
pick,0.083858,0.000084,0.000141,0.000164,0.000154
series,0.023903,0.000085,0.000140,0.000163,0.006976
round,0.017082,0.000084,0.000139,0.000164,0.000773
fund,0.014564,0.000084,0.000141,0.000164,0.000229
valuation,0.013033,0.000084,0.000140,0.000164,0.006249
today,0.012404,0.001058,0.000140,0.000164,0.000154


In [10]:
#find out the dominant topic
lda_coursera_output = lda_coursera_corpus.transform(bow_coursera_corpus)
doc_names = ["Doc_" + str(i) for i in range(len(normalized_coursera))]
topic_names = ["Topic_" + str(i) for i in range(no_topics)]
df_document_topic = pd.DataFrame(np.round(lda_coursera_output, 4), columns=topic_names, index=doc_names)
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
df_document_topic[0:4]

#find out which this belongs to 

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,dominant_topic
Doc_0,0.0222,0.0222,0.9109,0.0222,0.0224,2
Doc_1,0.0857,0.0857,0.6570,0.0857,0.0857,2
Doc_2,0.8153,0.0462,0.0462,0.0462,0.0462,0
Doc_3,0.0263,0.1401,0.0262,0.0263,0.7812,4


In [11]:
pyLDAvis.enable_notebook()
visualization_panel = pyLDAvis.sklearn.prepare(lda_coursera_corpus, bow_coursera_corpus, bow_vectorizer_coursera, mds='tsne')
visualization_panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      147.469925 -163.068634       1        1  24.481201
3      129.967133  244.420135       2        1  23.270402
0      -57.230949  -75.247375       3        1  22.038843
4      -66.374695  139.241669       4        1  16.528342
2      207.709900   43.632816       5        1  13.681213, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
954   Default  225.000000       startup  225.000000  30.0000  30.0000
584   Default  243.000000         learn  243.000000  29.0000  29.0000
1090  Default  179.000000         value  179.000000  28.0000  28.0000
771   Default  175.000000          pick  175.000000  27.0000  27.0000
729   Default  314.000000        online  314.000000  26.0000  26.0000
386   Default   97.000000         first   97.000000  25.0000  25.0000
1041  Default   94.000000          time   94.000000  24.0000  24.0000
130   Default   89.000000        campus   89.000000  23.0000  23.0000
608   Default   87.000000          long   87.000000  22.0000  22.0000
747   Default   86.000000       overdue   86.000000  21.0000  21.0000
842   Default   86.000000      remember   86.000000  20.0000  20.0000
859   Default   86.000000      revision   86.000000  19.0000  19.0000
1129  Default   86.000000           wit   86.000000  18.0000  18.0000
636   Default   86.000000         maths   86.000000  17.0000  17.0000
922   Default   85.000000         smith   85.000000  16.0000  16.0000
654   Default   74.000000       million   74.000000  15.0000  15.0000
823   Default   99.000000         raise   99.000000  14.0000  14.0000
585   Default   59.000000       learner   59.000000  13.0000  13.0000
401   Default   54.000000        fourth   54.000000  12.0000  12.0000
790   Default   54.000000       prepare   54.000000  11.0000  11.0000
861   Default   53.000000    revolution   53.000000  10.0000  10.0000
499   Default   51.000000    industrial   51.000000   9.0000   9.0000
896   Default   58.000000        series   58.000000   8.0000   8.0000
247   Default   51.000000   datascience   51.000000   7.0000   7.0000
216   Default  165.000000        course  165.000000   6.0000   6.0000
77    Default   44.000000  astrobiology   44.000000   5.0000   5.0000
206   Default   43.000000          cool   43.000000   4.0000   4.0000
609   Default   39.000000          look   39.000000   3.0000   3.0000
404   Default   73.000000          free   73.000000   2.0000   2.0000
180   Default   42.000000     colleague   42.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
189    Topic5   12.689130     component   13.065174   1.9599  -4.3906
498    Topic5   12.223947         india   12.599547   1.9589  -4.4280
83     Topic5   11.340715          auto   11.716061   1.9566  -4.5030
908    Topic5   11.340673        shweta   11.716331   1.9566  -4.5030
142    Topic5    9.481168      catalyst    9.857542   1.9502  -4.6821
184    Topic5   13.893230     community   14.466180   1.9487  -4.3000
987    Topic5    9.120955       success    9.509237   1.9475  -4.7208
802    Topic5    8.950864       program    9.347534   1.9458  -4.7396
576    Topic5    7.279924        launch    7.662581   1.9379  -4.9463
759    Topic5    7.018869          path    7.408064   1.9352  -4.9828
1011   Topic5    6.646666         teach    7.029446   1.9332  -5.0373
1116   Topic5    6.355013           way    6.732318   1.9315  -5.0821
751    Topic5    5.945059         owner    6.321128   1.9278  -5.1488
997    Topic5    5.889922   sustainable    6.265546   1.9273  -5.1581
393    Topic5    5.888786          food    6.265683   1.9271  -5.1583
377    Topic5    4.131341    fellowship    4.511963   1.9010  -5.5128
237    Topic5    3.884423        cursos    4.259711   1.8969  -5.5744
397    Topic5    3.884939     formacion    4.260518

**** LOG-LIKELIHOOD, PERPLEXITY AND COHERENCE SCORES ****

In [46]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
C:\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [47]:
coursera_corpus_tokenized = [tokenize_text(normalized_coursera[doc_id]) for doc_id in range(len(normalized_coursera))]
coursera_dictionary = Dictionary(coursera_corpus_tokenized)
coursera_corpus_bow = [coursera_dictionary.doc2bow(doc) for doc in coursera_corpus_tokenized]

In [50]:
cm = CoherenceModel(topics=topic_words, 
                    corpus = coursera_corpus_bow , 
                    dictionary = coursera_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(),4))  # get coherence value
print("Coherence score by topic: ", np.round(cm.get_coherence_per_topic(),4)) #coherence score by topic

Coherence score for the model:  -6.6168
Coherence score by topic:  [ -4.8323  -0.0441  -7.1457 -10.928  -10.134 ]


In [52]:
print("Log-Likelihood (higher values are better): ", lda_coursera_corpus.score(bow_coursera_corpus))

Log-Likelihood (higher values are better):  -43179.688970068324


In [53]:
print("Perplexity (lower values are better): ", lda_coursera_corpus.perplexity(bow_coursera_corpus))

Perplexity (lower values are better):  222.64297410156001


In [80]:
def select_highest_loglikelihood(lowest, highest):
    candidate = []
    for i in range(lowest,highest+1):
        candidate.append(i)
    no_keywords = 10
    res = []
    for i in range(lowest,highest+1):
        no_topics = i    
        lda_coursera_corpus = LatentDirichletAllocation(n_components=no_topics, max_iter=100,random_state = 42).fit(bow_coursera_corpus)
        res.append(lda_coursera_corpus.score(bow_coursera_corpus))
    idx = res.index(max(res))
    return candidate[idx]
def select_lowest_perplexity(lowest, highest):
    candidate = []
    for i in range(lowest,highest+1):
        candidate.append(i)
    no_keywords = 10
    res = []
    for i in range(lowest,highest+1):
        no_topics = i    
        lda_coursera_corpus = LatentDirichletAllocation(n_components=no_topics, max_iter=100,random_state = 42).fit(bow_coursera_corpus)
        res.append(lda_coursera_corpus.perplexity(bow_coursera_corpus))
    idx = res.index(min(res))
    return candidate[idx]
def select_highest_coherence_score(lowest, highest):
    candidate = []
    for i in range(lowest,highest+1):
        candidate.append(i)
    no_keywords = 10
    res = []
    for i in range(lowest,highest+1):
        no_topics = i    
        lda_coursera_corpus = LatentDirichletAllocation(n_components=no_topics, max_iter=100,random_state = 42).fit(bow_coursera_corpus)
        topic_words = get_topic_words(vectorizer = bow_vectorizer_coursera, 
                              lda_model = lda_coursera_corpus, 
                              n_words = no_keywords)
        cm = CoherenceModel(topics=topic_words, corpus = coursera_corpus_bow , dictionary = coursera_dictionary, coherence='u_mass')
        res.append(np.round(cm.get_coherence(),4))
    idx = res.index(max(res))
    return candidate[idx]

In [64]:
select_highest_loglikelihood(2,5)

C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  Deprecat

5

In [68]:
select_lowest_perplexity(2,5)

C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  Deprecat

5

In [79]:
select_highest_coherence_score(2, 5)

C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  Deprecat

(5, [-11.9828, -10.4517, -9.6755, -8.359])